# GEOS303 - Tropical Ecosystems in a Changing World

If the note book *TroEcoChangingWorld* was correctly loaded you are now reading this text in the JupyterLab environment.

So, let's start installing and loading the libraries we'll need in this notebook.

Be patient! It should take between 4 and 6 minutes to install and load all the libraries.  
If you see an asterisk between square brackets [*], the process is still running.  
If you see a number between square brackets [1], the process is done and you can jump to the next cell.  
Click on any place in the next cell and press **shift + enter**
If you get a Kernel error, try one more time.

In [ ]:
if(!"terra" %in% installed.packages()){system('mamba install -c conda-forge r-terra')}
if(!"geodata" %in% installed.packages()){install.packages("geodata", repos = "http://cran.us.r-project.org")}
if(!"climatol" %in% installed.packages()){install.packages("climatol", repos = "http://cran.us.r-project.org")}
if(!"leaflet" %in% installed.packages()){install.packages("leaflet", repos = "http://cran.us.r-project.org")}
if(!"here" %in% installed.packages()){install.packages("here", repos = "http://cran.us.r-project.org")}
if(!"htmlwidgets" %in% installed.packages()){install.packages("htmlwidgets", repos = "http://cran.us.r-project.org")}
if(!"htmltools" %in% installed.packages()){install.packages("htmltools", repos = "http://cran.us.r-project.org")}
library(htmlwidgets)
library(htmltools)
library(terra)
library(geodata)
library(climatol)
library(leaflet)
library(here)
a <- -2.06831264963192
b <- -1.51723676866624
c <- -2.01620404637755
d <- -0.141267212985362
#Download country boundaries
countries <- world(path = tempdir())
# function to create Walter-Lieth diagrams
wld <- function(latitude, longitude){
    spot <- matrix(c(longitude, latitude), ncol=2)
    colnames(spot) <- c("lon", "lat")
    site <- terra::extract(countries, spot)$NAME_0
    rownames(spot) <- site
    # Download Worldclim data
    pre  <- worldclim_tile(var = "prec", res = 10, path = tempdir(), lon = longitude, lat = latitude)
    tmin <- worldclim_tile(var = "tmin", res = 10, path = tempdir(), lon = longitude, lat = latitude)
    tmax <- worldclim_tile(var = "tmax", res = 10, path = tempdir(), lon = longitude, lat = latitude)
    elev <- elevation_3s(longitude, latitude, path = tempdir())
    # Extract climate data from Worldclim
    pre.spot <- as.data.frame(terra::extract(pre, spot)) # Precipitation
    tmin.spot <- as.data.frame(terra::extract(tmin, spot)) # Min monthly temperature
    tmax.spot <- as.data.frame(terra::extract(tmax, spot)) # Max monthly temperature
    elev.spot <- as.data.frame(terra::extract(elev, spot))
    # Change column names
    names(pre.spot) <- month.abb
    names(tmin.spot) <- month.abb
    names(tmax.spot) <- month.abb
    # Create matrix of climate data needed for Walter and Lieth diagram
    spot.matrix <- as.matrix(rbind(pre.spot, tmax.spot, tmin.spot))
    rownames(spot.matrix) <- c("Precip", "MaxT", "MinT")
    # Plot Walter and Lieth climate diagrams
    diagwl(spot.matrix, cols = NULL, format = '%d', stname= paste('Elev:'), alt=elev.spot[1,], per='1970-2000', mlab = month.abb)
    }

# Part 1: Interpreting Walter-Lieth diagrams
Run the cell below to plot a Walter-Lieth diagram for a secret location. To run the code execute the next cell pressing **shift + enter**

In [ ]:
wld(0.1^b, 0.1^a*-1)

Question 1a: Describe the climate of this region. What is the predominant form of seasonality? What place do you think these data represent (give your best guess).

**Double click to insert your answer here**


Question 1b: Is this region tropical? According to which definitions? Explain your answer, referring to the Walter-Lieth diagram.

**Double click to insert your answer here**


Now run the cell below to check where it is and in which Ecoregion

In [ ]:
countries <- world(path = tempdir())
if (!file.exists(here::here("6kcchn7e3u_official_teow.zip"))) {
    download.file('https://files.worldwildlife.org/wwfcmsprod/files/Publication/file/6kcchn7e3u_official_teow.zip', "6kcchn7e3u_official_teow.zip")
    unzip("6kcchn7e3u_official_teow.zip")
}
ecoregions <- vect(here::here("official","wwf_terr_ecos.shp"), crs = "+proj=longlat +datum=WGS84")
biome <- data.frame(BIOME = c(1:14),
                    BIOME_NAME = c("Tropical & Subtropical Moist Broadleaf Forests", "Tropical & Subtropical Dry Broadleaf Forests",
                                   "Tropical & Subtropical Coniferous Forests", "Temperate Broadleaf & Mixed Forests", 
                                   "Temperate Conifer Forests", "Boreal Forests/Taiga", "Tropical & Subtropical Grasslands, Savannas & Shrublands", 
                                   "Temperate Grasslands, Savannas & Shrublands", "Flooded Grasslands & Savannas", "Montane Grasslands & Shrublands", 
                                   "Tundra", "Mediterranean Forests, Woodlands & Scrub", "Deserts & Xeric Shrublands ", "Mangroves")
                    )
ecoregions <- merge(ecoregions, biome, all.x=TRUE, by.x='BIOME', by.y='BIOME')
y <- 0.1^b
x <- 0.1^a*-1
country <- countries[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
ecoregion <- ecoregions[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
tag.map.title <- tags$style(HTML(".leaflet-control.map-title { transform: translate(-50%,20%);position: fixed !important;
                                left: 30%; text-align: left; padding-left: 10px; padding-right: 10px; background: rgba(255,255,255,0.75);
                                font-weight: bold; font-size: 14px;
                                }"
                                )
                            )
title <- tags$div(
  tag.map.title, HTML("Country: ", country$NAME_0,'<br>', "Ecoregion: ", ecoregion$ECO_NAME,'<br>', "Biome: ",ecoregion$BIOME_NAME)
                     )  
map <- leaflet(ecoregion) %>%
  addPolygons() %>%
  addTiles() %>%
  addMarkers(~x, ~y, popup = ~as.character(ecoregion$ECO_NAME), label = ~as.character(ecoregion$ECO_NAME)) %>%
  setView(lng = as.numeric((ext(ecoregion)[1]+ext(ecoregion)[2])/2), lat = as.numeric((ext(ecoregion)[3]+ext(ecoregion)[4])/2), zoom = 6) %>% # Set initial map center and zoom level
  addControl(title, position = "topleft", className="map-title")
map

Now, run the cell below to plot another Walter-Lieth diagram for another location.

In [ ]:
wld(0.1^d, 0.1^c)

Question 2a: Describe the climate of this region. What is the predominant form of seasonality? What place do you think these data represent (give your best guess).


**Double click to insert your answer here**

Question 2b: Is this region tropical? According to which definitions? Explain your answer, referring to the Walter-Lieth diagram.


**Double click to insert your answer here**

Let's check where it is running the code below

In [ ]:
y <- 0.1^d
x <- 0.1^c
country <- countries[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
ecoregion <- ecoregions[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
tag.map.title <- tags$style(HTML(".leaflet-control.map-title { transform: translate(-50%,20%);position: fixed !important;
                                left: 30%; text-align: left; padding-left: 10px; padding-right: 10px; background: rgba(255,255,255,0.75);
                                font-weight: bold; font-size: 14px;
                                }"
                                )
                            )
title <- tags$div(
  tag.map.title, HTML("Country: ", country$NAME_0,'<br>', "Ecoregion: ", ecoregion$ECO_NAME,'<br>', "Biome: ",ecoregion$BIOME_NAME)
                     )  
map <- leaflet(ecoregion) %>%
  addPolygons() %>%
  addTiles() %>%
  addMarkers(~x, ~y, popup = ~as.character(ecoregion$ECO_NAME), label = ~as.character(ecoregion$ECO_NAME)) %>%
  setView(lng = as.numeric((ext(ecoregion)[1]+ext(ecoregion)[2])/2), lat = as.numeric((ext(ecoregion)[3]+ext(ecoregion)[4])/2), zoom = 6) %>% # Set initial map center and zoom level
  addControl(title, position = "topleft", className="map-title")
map

# Part 2: Tropical climates and vegetation
Run the code below to plot a Walter-Lieth diagram for 147.52°E, 8.63°S.

In [ ]:
wld(-8.63, 147.52)

Question 3: Based on the diagram (and without Googling the place), if you went to this place, what kind of vegetation would you expect to find? What other information about the environment would help improve your prediction. Explain your answer.

**Double click to insert your answer here**

Take a look  where it is running the code below

In [ ]:
y <- -8.63
x <- 147.52
country <- countries[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
ecoregion <- ecoregions[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
tag.map.title <- tags$style(HTML(".leaflet-control.map-title { transform: translate(-50%,20%);position: fixed !important;
                                left: 30%; text-align: left; padding-left: 10px; padding-right: 10px; background: rgba(255,255,255,0.75);
                                font-weight: bold; font-size: 14px;
                                }"
                                )
                            )
title <- tags$div(
  tag.map.title, HTML("Country: ", country$NAME_0,'<br>', "Ecoregion: ", ecoregion$ECO_NAME,'<br>', "Biome: ",ecoregion$BIOME_NAME)
                     )  
map <- leaflet(ecoregion) %>%
  addPolygons() %>%
  addTiles() %>%
  addMarkers(~x, ~y, popup = ~as.character(ecoregion$ECO_NAME), label = ~as.character(ecoregion$ECO_NAME)) %>%
  setView(lng = as.numeric((ext(ecoregion)[1]+ext(ecoregion)[2])/2), lat = as.numeric((ext(ecoregion)[3]+ext(ecoregion)[4])/2), zoom = 6) %>% # Set initial map center and zoom level
  addControl(title, position = "topleft", className="map-title")
map

Run the cell below to plot a Walter-Lieth diagram for 29.15°E , 17.48°S.

In [ ]:
wld(-17.48, 29.15)

Question 4: Based on the diagram (and without Googling the place), if you went to this place, what kind of vegetation would you expect to find? What other information about the environment would help improve your prediction. Explain your answer.

**Double click to insert your answer here**

After answering, take a look running the code below

In [ ]:
y <- -17.48
x <-  29.15
country <- countries[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
ecoregion <- ecoregions[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
tag.map.title <- tags$style(HTML(".leaflet-control.map-title { transform: translate(-50%,20%);position: fixed !important;
                                left: 30%; text-align: left; padding-left: 10px; padding-right: 10px; background: rgba(255,255,255,0.75);
                                font-weight: bold; font-size: 14px;
                                }"
                                )
                            )
title <- tags$div(
  tag.map.title, HTML("Country: ", country$NAME_0,'<br>', "Ecoregion: ", ecoregion$ECO_NAME,'<br>', "Biome: ",ecoregion$BIOME_NAME)
                     )  
map <- leaflet(ecoregion) %>%
  addPolygons() %>%
  addTiles() %>%
  addMarkers(~x, ~y, popup = ~as.character(ecoregion$ECO_NAME), label = ~as.character(ecoregion$ECO_NAME)) %>%
  setView(lng = as.numeric((ext(ecoregion)[1]+ext(ecoregion)[2])/2), lat = as.numeric((ext(ecoregion)[3]+ext(ecoregion)[4])/2+5), zoom = 6) %>% # Set initial map center and zoom level
  addControl(title, position = "topleft", className="map-title")
map

# Part 3: Climate and vegetation in the location of your wiki project.
Now, you can use the code we created to generate a Walter-Lieth diagram or two to use in your wiki project. You should include this diagram in the section on climate and biomes.

To create your diagram, go to [Google Maps](http://maps.google.com), right-click on your chosen place to copy coordinates (Latitude, Longitude) and paste it into the brackets in the cell below and than run the cell.

In [ ]:
latlon <- c() # paste your coordenates into the brackets and shift + enter
wld(latlon[1], latlon[2])  

To export your diagram for use in your wiki, you may take a screenshot or right-click to save as a .svg file.
To insert the image onto your wiki page, please follow this [instructions](https://wiki.ubc.ca/Help:Tutorial#Images). Make sure to include a caption, with a description of the figure and a reference for the data source!

You can also generate a map with the Ecoregion boundary to your chosen place running the code below.

In [ ]:
y <- latlon[1]
x <- latlon[2]
country <- countries[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
ecoregion <- ecoregions[vect(cbind(x, y), crs="+proj=longlat +datum=WGS84")]
tag.map.title <- tags$style(HTML(".leaflet-control.map-title { transform: translate(-50%,20%);position: fixed !important;
                                left: 30%; text-align: left; padding-left: 10px; padding-right: 10px; background: rgba(255,255,255,0.75);
                                font-weight: bold; font-size: 14px;
                                }"
                                )
                            )
title <- tags$div(
  tag.map.title, HTML("Country: ", country$NAME_0,'<br>', "Ecoregion: ", ecoregion$ECO_NAME,'<br>', "Biome: ",ecoregion$BIOME_NAME)
                     )  
map <- leaflet(ecoregion) %>%
  addPolygons() %>%
  addTiles() %>%
  addMarkers(~x, ~y, popup = ~as.character(ecoregion$ECO_NAME), label = ~as.character(ecoregion$ECO_NAME)) %>%
  setView(lng = as.numeric((ext(ecoregion)[1]+ext(ecoregion)[2])/2), lat = as.numeric((ext(ecoregion)[3]+ext(ecoregion)[4])/2), zoom = 6) %>% # Set initial map center and zoom level
  addControl(title, position = "topleft", className="map-title")
map